In [62]:
%matplotlib inline 
from nltk.collocations import *
from gensim.models import word2vec
import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product
from __future__ import division
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import nltk, string
from nltk.tokenize import punkt
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pymorphy2
import sys
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import  roc_auc_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import ward, dendrogram
from scipy.cluster import hierarchy
from sklearn.metrics.pairwise import cosine_similarity
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
# exclude = set(punctuation + u'0123456789[]—«»–')
exclude = set(punctuation + u'[]—«»–')
# import pylanguagetool
import requests
import json
from gensim.models.keyedvectors import KeyedVectors
import xgboost as xgb

In [63]:
re1 = re.compile("""
    (?:
        (?:
            (?<!\\d(?:р|г|к))
            (?<!и\\.т\\.(?:д|п))
            (?<!и(?=\\.т\\.(?:д|п)\\.))
            (?<!и\\.т(?=\\.(?:д|п)\\.))
            (?<!руб|коп)
        \\.) |
        [!?\\n]
    )+
    """, re.X)

def safe_split(regex, text):
    res = []
    sear = regex.search(text)
    while sear:
        res.append(text[:sear.end()])
        text = text[sear.end():]
        sear = regex.search(text)
    res.append(text)
    return res

In [3]:
def get_patterns(text):
    tokens = text.split(' ')
    return [[p.tag.POS for p in morph.parse(word)][0] for word in tokens]

In [4]:
pd.set_option('display.max_columns', 100)

In [5]:
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [178]:
def normalize_answer(text):
    return ' '.join(re.findall(r"\w+", text)).lower()

def normalize_answer_no_lower(text):
    return ' '.join(re.findall(r"\w+", text))

def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(normalize_answer(sentence).split())
    return sentences_in_words

def text_to_sentence(text):
    sentences = safe_split(re1, text)
    return [s.strip() for s in sentences if s.strip() != '']

def get_max_match_sentance(data_row, w2v = False):
    sentences = text_to_sentence(data_row["paragraph"])
    sentences_in_words = sentence_to_word(sentences)
    question_in_words = sentence_to_word([data_row["question"]])[0]
    
    max_overlap = None
    max_match_sentance_id = None
    
    overlaps = []
    question_words = set(question_in_words)
    for sentance_id in range(len(sentences_in_words)):
        sentence_words = set(sentences_in_words[sentance_id])
        if w2v:
            overlap = texts_intersection(sentences_in_words[sentance_id], question_in_words, model, 0.2)
        else:
            overlap = sentence_words.intersection(question_words)
        overlaps.append(overlap)
        
        overlap_len = len(overlap)
        if max_overlap is None or overlap_len > max_overlap:
            max_overlap = overlap_len
            max_match_sentance_id = sentance_id
            
    prediction_sentence = sentences[max_match_sentance_id]
    prediction_words = ' '.join([x for x in prediction_sentence.split(' ') if x not in data_row["question"].split(' ')])
    question_residuals = ' '.join([x for x in data_row["question"].split() if x not in prediction_sentence.split()])

    return overlaps, prediction_sentence, prediction_words, question_residuals

In [7]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter = Counter()
    uniq_paragraphs = data['paragraph'].unique()
    uniq_questions = data['question'].unique()
    for paragraph in tqdm.tqdm(uniq_paragraphs, desc="calc idf for paragraph"):
        set_words = uniq_words(paragraph)
        counter.update(set_words)
    for question in tqdm.tqdm(uniq_questions, desc="calc idf for question"):
        set_words = uniq_words(question)
        counter.update(set_words)
    num_docs = uniq_paragraphs.shape[0] + uniq_questions.shape[0]
    idfs = {}
    for word in counter:
        idfs[word] = np.log(num_docs / counter[word])
    return idfs

In [8]:
def preprocess(text):
    
    buf = ''.join(ch for ch in text if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(buf.lower())
    lemmatizer = MorphAnalyzer()
    lemmas = []
    for t in tokens[:]:
        if not t in stopwords.words('russian'):
            try:
                lemma = lemmatizer.parse(t)[0].normal_form
            except: 
                lemma = t
            lemmas.append(lemma)
    return ' '.join(lemmas)

In [9]:
def texts_intersection(text1, text2, word2vec, threshold):
    words1 = set(text1)
    words2 = set(text2)
    result = []
    for word1 in words1:
        for word2 in words2:
            if word1 == word2:
                similarity = 1.0
            elif word1 in word2vec and word2 in word2vec:
                similarity = word2vec.similarity(word1, word2)
            else:
                similarity = 0.0
            if similarity >= threshold:
                result.append(word1)
                result.append(word2)
    return ' '.join(result)

In [10]:
patterns = ['ADJF+NOUN','PRTS+NOUN','NOUN+NOUN', 'NOUN+VERB', 'VERB+ADJF', 'VERB+NOUN']
morph = pymorphy2.MorphAnalyzer()
def match(ngram, patterns = patterns):
    index = []
    for word in ngram:
        buf = [(p.normal_form, p.tag.POS) for p in morph.parse(word)]
        index.append((word,buf))
    pos_tagging = product(*[ind[1] for ind in index])
    possible_patterns = map(lambda pos_tag: list(zip(*pos_tag)), pos_tagging)
    possible_patterns = map(lambda pattern: [pattern[0], map(lambda grammeme: grammeme, pattern[1])], possible_patterns)
    possible_patterns = map(lambda pattern: (pattern[0], '+'.join(pattern[1])), possible_patterns)
    for pattern in possible_patterns:
        if pattern[1] in patterns:
            return pattern
    return None        

def get_pattern(text):
    tokens = [x for x in normalize(text)]
    pattern_coll2  = nltk.FreqDist() 
    coll2 = nltk.FreqDist(nltk.bigrams(tokens))
    for c2 in coll2:
        try:
            p = match(c2)
            if p != None:
                collocation = ' '.join(p[0])
                if collocation in pattern_coll2:
                    pattern_coll2[collocation] += coll2[c2]
                else:
                    pattern_coll2[collocation] = coll2[c2]
        except:
            pass
    return pattern_coll2

## Data

In [ ]:
372	27273

In [173]:
df = pd.read_csv("../../data/sdsj_A_test.csv")
df[df['paragraph_id'] == 372][df['question_id'] == 27273]

,paragraph_id,question_id,paragraph,question
66135,372,27273,"Международный арбитражный суд при Международной торговой палате (англ. International Court of Arbitration of the International Chamber of Commerce) — международный суд, разрешающий арбитражные коммерческие споры. Основан в 1923 году, министром финансов Франции Этьеном Клементелем. Выступает в роли третейского посредника в разрешении споров. Решения суда не имеют обязательной силы. Формирование состава суда происходит по общему правилу третейских судов. Наиболее известный и опытный среди международных арбитражных институтов.",Как происходит формирование состава суда?


In [72]:
dftrain, dftest = pd.read_csv("../../data/train_task1_latest.csv"), pd.read_csv("../../data/sdsj_A_test.csv")

In [ ]:
# # add pattern generated questions
# dftrain_generated = pd.read_csv("../../data/taskA_generated_questions_p.csv", sep=';')
# dftrain_generated = dftrain_generated.sample(5000)
# question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
# for row in dftrain_generated.iterrows():
#     dftrain_generated.loc[row[0], 'question_id'] = question_id
#     question_id += 1
# dftrain_generated['target'] = 0.0
# dftrain_generated['generated_p'] = 1.0
# dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p']
# dftrain['generated_p'] = 0.0
# dftrain = dftrain.append(dftrain_generated, ignore_index=True)
# dftrain.fillna(0, inplace = True)

In [ ]:
# # add pattern_2 generated questions
# dftrain_generated = pd.read_csv("../../data/taskA_generated_questions_p_2.csv", sep=';')
# dftrain_generated = dftrain_generated.sample(5000)
# question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
# for row in dftrain_generated.iterrows():
#     dftrain_generated.loc[row[0], 'question_id'] = question_id
#     question_id += 1
# dftrain_generated['target'] = 0.0
# dftrain_generated['generated_p_2'] = 1.0
# dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_2']
# dftrain['generated_p_2'] = 0.0
# dftrain = dftrain.append(dftrain_generated, ignore_index=True)
# dftrain.fillna(0, inplace = True)

In [73]:
# add pattern_3 generated questions
dftrain_generated = pd.read_csv("../../data/taskA_generated_questions_p_3.csv", sep=';')
dftrain_generated = dftrain_generated.sample(20000)
question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
for row in dftrain_generated.iterrows():
    dftrain_generated.loc[row[0], 'question_id'] = question_id
    question_id += 1
dftrain_generated['target'] = 0.0
dftrain_generated['generated_p_3'] = 1.0
dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_3']
dftrain['generated_p_3'] = 0.0
dftrain = dftrain.append(dftrain_generated, ignore_index=True)
dftrain.fillna(0, inplace = True)

In [ ]:
# # add pattern_4 generated questions
# dftrain_generated = pd.read_csv("../../data/taskA_generated_questions_p_4.csv", sep=';')
# dftrain_generated = dftrain_generated.sample(5000)
# question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
# for row in dftrain_generated.iterrows():
#     dftrain_generated.loc[row[0], 'question_id'] = question_id
#     question_id += 1
# dftrain_generated['target'] = 0.0
# dftrain_generated['generated_p_4'] = 1.0
# dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_4']
# dftrain['generated_p_4'] = 0.0
# dftrain = dftrain.append(dftrain_generated, ignore_index=True)
# dftrain.fillna(0, inplace = True)

In [74]:
# add pattern_6 generated questions
dftrain_generated = pd.read_csv("../../data/taskA_generated_questions_p_6.csv", sep=';')
dftrain_generated = dftrain_generated
question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
for row in dftrain_generated.iterrows():
    dftrain_generated.loc[row[0], 'question_id'] = question_id
    question_id += 1
dftrain_generated['target'] = 0.0
dftrain_generated['generated_p_6'] = 1.0
dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_6']
dftrain['generated_p_6'] = 0.0
dftrain = dftrain.append(dftrain_generated, ignore_index=True)
dftrain.fillna(0, inplace = True)

## Baseline

In [75]:
idfs = calculate_idfs(dftrain.append(dftest, ignore_index = True))

calc idf for question: 100%|██████████| 122193/122193 [00:01<00:00, 63552.35it/s]


In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        question = uniq_words(row.question)
        paragraph = uniq_words(row.paragraph)
        df.loc[index, 'len_paragraph'] = len(paragraph)
        df.loc[index, 'len_question'] = len(question)
        df.loc[index, 'len_intersection'] = len(paragraph & question)
        df.loc[index, 'idf_question'] = np.sum([idfs.get(word, 0.0) for word in question])
        df.loc[index, 'idf_paragraph'] = np.sum([idfs.get(word, 0.0) for word in paragraph])
        df.loc[index, 'idf_intersection'] = np.sum([idfs.get(word, 0.0) for word in paragraph & question])
        
    df['relative_question_len'] = df['len_question'] / df['len_paragraph']
    df['relative_intersection_len'] = df['len_intersection'] / df['len_paragraph']
    df['relative_intersection_question_len'] = df['len_intersection'] / df['len_question']
    
    df['relative_question_idf'] = df['idf_question'] / df['idf_paragraph']
    df['relative_intersection_idf'] = df['idf_intersection'] / df['idf_paragraph']
    df['relative_intersection_question_idf'] = df['idf_intersection'] / df['idf_question']
    
    df['word_idf_paragraph'] = df['idf_paragraph'] / df['len_paragraph']
    df['word_idf_question'] = df['idf_question'] / df['len_question']
    df['word_idf_intersection'] = df['idf_intersection'] / df['len_intersection'] 
    
    df.fillna(0, inplace = True)

build features for test: 100%|██████████| 74286/74286 [04:49<00:00, 256.46it/s]


## Data mining


**lemmarize**

In [ ]:
%%time
# train - paragraph
df = pd.DataFrame(dftrain.paragraph.unique())
df.columns = ['paragraph']
df['paragraph_lemmatized'] = df['paragraph'].map(lambda x: preprocess(x))
dftrain = dftrain.merge(df, on = 'paragraph', how = 'left')
print('dftrain complete - paragraph_lemmatized')

# train - question
df = pd.DataFrame(dftrain.question.unique())
df.columns = ['question']
df['question_lemmatized'] = df['question'].map(lambda x: preprocess(x))
dftrain = dftrain.merge(df, on = 'question', how = 'left')
print('dftrain complete - question_lemmatized')

# test - paragraph
df = pd.DataFrame(dftest.paragraph.unique())
df.columns = ['paragraph']
df['paragraph_lemmatized'] = df['paragraph'].map(lambda x: preprocess(x))
dftest = dftest.merge(df, on = 'paragraph', how = 'left')
print('dftest complete - paragraph_lemmatized')

# test - question
df = pd.DataFrame(dftest.question.unique())
df.columns = ['question']
df['question_lemmatized'] = df['question'].map(lambda x: preprocess(x))
dftest = dftest.merge(df, on = 'question', how = 'left')
print('dftest complete - question_lemmatized')

dftrain complete - paragraph_lemmatized
dftrain complete - question_lemmatized


**word2vec similarity**

In [ ]:
# load model

model = KeyedVectors.load_word2vec_format("../../data/ruwikiruscorpora_0_300_20.bin.gz", binary=True)
new_vocab = {}
[new_vocab.update({k[0].replace("::", "_").split('_')[0]: k[1]}) for k in model.vocab.items()]
model.vocab = new_vocab

# or build model

# model = word2vec.Word2Vec(np.append(dftrain.paragraph_lemmatized, 
#                                     np.append(dftrain[dftrain.generated_p_2 ==0][dftrain.generated_p ==0].question_lemmatized, 
#                                               dftest.paragraph_lemmatized)),
#                  size=100, 
#                  window=5, 
#                  min_count=5, 
#                  workers=4)
# fname = '../results/word2vec_model'
# model.save(fname)
# # model = word2vec.Word2Vec.load(fname)

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word2vec"):  
        question = row['question_lemmatized']
        paragraph = row['paragraph_lemmatized']
        w2v_similarities = []
        for question_word in question.split(' '):
            max_similarity = 0.0
            for paragraph_word in paragraph.split(' '):
                if (question_word in model.wv.vocab) and (paragraph_word in model.wv.vocab):
                    similarity = model.similarity(question_word, paragraph_word)
                    if similarity > max_similarity:
                        max_similarity = similarity
            w2v_similarities.append(max_similarity)
#         w2v_similarities = [x for x in w2v_similarities if x > 0.2]
        
        if len(w2v_similarities) == 0:
            df.loc[index, 'word2vec_similarity_max'] = 0
            df.loc[index, 'word2vec_similarity_min'] = 0
        else:
            df.loc[index, 'word2vec_similarity_max'] = np.max(w2v_similarities)
            df.loc[index, 'word2vec_similarity_min'] = np.min(w2v_similarities)
        df.loc[index, 'word2vec_similarity_mean'] = np.mean(w2v_similarities)
        df.loc[index, 'word2vec_similarity_std'] = np.std(w2v_similarities)
    df.fillna(0, inplace = True)

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        intersection = texts_intersection(row.paragraph_lemmatized.split(' '), row.question_lemmatized.split(' '), model, 0.2)
        df.loc[index, 'len_intersection_w2v'] = len(intersection) / 2.0
        df.loc[index, 'idf_intersection_w2v'] = np.sum([idfs.get(word, 0.0) for word in intersection]) / 2.0
    df['relative_intersection_len_w2v'] = df['len_intersection_w2v'] / df['len_paragraph']
    df['relative_intersection_question_len_w2v'] = df['len_intersection_w2v'] / df['len_question']
    df['relative_intersection_idf_w2v'] = df['idf_intersection_w2v'] / df['idf_paragraph']
    df['relative_intersection_question_idf_w2v'] = df['idf_intersection_w2v'] / df['idf_question']
    df['word_idf_intersection_w2v'] = df['idf_intersection_w2v'] / df['len_intersection'] 
    df.fillna(0, inplace = True)

**word mover distance**

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        df.loc[index, 'word_mover_distance'] = model.wmdistance(row['paragraph'], row['question'])
        df.loc[index, 'word_mover_distance_lemm'] = model.wmdistance(row['paragraph_lemmatized'], row['question_lemmatized'])
        
        wmds = []
        for paragraph_sentence in text_to_sentence(row['paragraph']):
            wmds.append(model.wmdistance(paragraph_sentence, row['question']))
            
        df.loc[index, 'word_mover_sentence_distance_mean'] = np.mean(wmds)
        df.loc[index, 'word_mover_sentence_distance_std'] = np.std(wmds)
        df.loc[index, 'word_mover_sentence_distance_max'] = np.max(wmds)
        df.loc[index, 'word_mover_sentence_distance_min'] = np.min(wmds)

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        question = row.question
        question_lemm = row.question_lemmatized
        question_len = len(question_lemm)
        similarities = []
        similarities_lemm = []
        if int(question_len / 3) != 0:
            for i in range(0, question_len, int(question_len / 3)):
                question_chunk = question[i:i + int(question_len / 3)]
                similarities.append(model.wmdistance(question_chunk, row['paragraph']))
                question_chunk_lemm = question_lemm[i:i + int(question_len / 3)]
                similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
        elif int(question_len / 2) != 0:
            for i in range(0, question_len, int(question_len / 2)):
                question_chunk = question[i:i + int(question_len / 2)]
                similarities.append(model.wmdistance(question_chunk, row['paragraph']))
                question_chunk_lemm = question_lemm[i:i + int(question_len / 2)]
                similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
        else:
            question_chunk = question
            similarities.append(model.wmdistance(question_chunk, row['paragraph']))
            question_chunk_lemm = question_lemm
            similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
            
        df.loc[index, 'word_mover_sentence_distance_split_max'] = np.max(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_min'] = np.min(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_mean'] = np.mean(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_std'] = np.std(similarities)
        
        df.loc[index, 'word_mover_sentence_distance_split_lemm_max'] = np.max(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_min'] = np.min(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_mean'] = np.mean(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_std'] = np.std(similarities_lemm)

**max match sentence**

In [181]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating max overlap"):        
        overlaps, prediction_sentence, answer, question_residuals = get_max_match_sentance(row)
        
        # vars
        a = [len(x) / len(row['paragraph']) for x in overlaps]
        b = [len(x) / len(row['question']) for x in overlaps]
        c = [np.sum([idfs.get(word, 0.0) for word in overlap]) for overlap in overlaps]
        
        # answer 
        df.loc[index, 'answer'] = answer
        
        # prediction_sentence
        df.loc[index, 'prediction_sentence'] = prediction_sentence
        
        # answer len
        answer_len = len(answer.split(' '))
        df.loc[index, 'answer_len'] = answer_len
        if len(row.question) != 0:
            df.loc[index, 'answer_len_relative_question'] = answer_len / len(row.question)
        if len(prediction_sentence) != 0:
            df.loc[index, 'answer_len_relative_prediction_sentence'] = answer_len / len(prediction_sentence)
        
        # answer idf
        answer_idf = np.sum([idfs.get(word, 0.0) for word in answer.split(' ')])
        df.loc[index, 'answer_idf'] = answer_idf 
        df.loc[index, 'answer_idf_relative_question'] = answer_idf / np.sum([idfs.get(word, 0.0) for word in row.question.split(' ')])
        df.loc[index, 'answer_idf_relative_prediction_sentence'] = answer_idf / np.sum([idfs.get(word, 0.0) for word in prediction_sentence.split(' ')])
        
        # residuals
        df.loc[index, 'question_residuals'] = question_residuals
        
        # residuals len
        question_residuals_len = len(question_residuals.split(' '))
        df.loc[index, 'question_residuals_len'] = question_residuals_len
        if len(row.question) != 0:
            df.loc[index, 'question_residuals_len_relative_question'] = question_residuals_len / len(row.question)
        if len(prediction_sentence) != 0:
            df.loc[index, 'question_residuals_len_relative_prediction_sentence'] = question_residuals_len / len(prediction_sentence)
            
        # residuals idf
        question_residuals_idf = np.sum([idfs.get(word, 0.0) for word in question_residuals.split(' ')])
        df.loc[index, 'question_residuals_idf'] = question_residuals_idf 
        df.loc[index, 'question_residuals_idf_relative_question'] = question_residuals_idf / np.sum([idfs.get(word, 0.0) for word in row.question.split(' ')])
        df.loc[index, 'question_residuals_idf_relative_prediction_sentence'] = question_residuals_idf / np.sum([idfs.get(word, 0.0) for word in prediction_sentence.split(' ')])
        
        # max
        max_overlap = ''
        max_overlap_idf = 0
        for overlap in overlaps:
            overlap_idf = np.sum([idfs.get(word, 0.0) for word in overlap])
            if overlap_idf > max_overlap_idf:
                max_overlap_idf = overlap_idf
                max_overlap = overlap
        df.loc[index, 'overlap'] = ' '.join(max_overlap)
        
        df.loc[index, 'len_overlap_relative_paragraph_max'] = np.max(a)
        df.loc[index, 'len_overlap_relative_question_max'] = np.max(b)
        df.loc[index, 'idf_overlap_max'] = np.max(c)
        
        # means
        df.loc[index, 'len_overlap_relative_paragraph_mean'] = np.mean(a)
        df.loc[index, 'len_overlap_relative_question_mean'] = np.mean(b)
        df.loc[index, 'idf_overlap_mean'] = np.mean(c)
        
        # stds
        df.loc[index, 'len_overlap_relative_paragraph_std'] = np.std(a)
        df.loc[index, 'len_overlap_relative_question_std'] = np.std(b)
        df.loc[index, 'idf_overlap_std'] = np.std(c)


calculating max overlap: 100%|██████████| 74286/74286 [37:57<00:00, 29.43it/s]


**similarity**

In [89]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectorizer.fit(set(np.append(pd.unique([x for x in dftrain.paragraph_lemmatized.values]),
                                     np.append(pd.unique([x for x in dftrain[dftrain.generated_p_3 ==0][dftrain.generated_p_6 ==0].question_lemmatized.values]),
                                              pd.unique([x for x in dftest.paragraph_lemmatized.values])))))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [90]:
# calculate cummulative nlpS
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
        vectors = vectorizer.transform([row.paragraph_lemmatized, row.question_lemmatized])
        df.loc[index, 'similarity_tfid'] = (vectors * vectors.T)[0, 1]

calculating nlps for test: 100%|██████████| 74286/74286 [08:33<00:00, 144.71it/s]


In [182]:
# word mover distance prediction
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        df.loc[index, 'word_mover_distance_prediction'] = model.wmdistance(row['prediction_sentence'], row['question'])

calculating word mover distance for test: 100%|██████████| 74286/74286 [05:07<00:00, 241.75it/s]


In [ ]:
# # calculate similarity between prediction sentence and question 
# for name, df in [('train', dftrain), ('test', dftest)]:
#     for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
#         vectors = vectorizer.transform([row.prediction_sentence, row.question])
#         df.loc[index, 'similarity_prediction_tfid'] = (vectors * vectors.T)[0, 1]

In [183]:
# word mover distance residuals
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        df.loc[index, 'word_mover_distance_residuals'] = model.wmdistance(row['question_residuals'], row['paragraph'])

calculating word mover distance for test: 100%|██████████| 74286/74286 [07:01<00:00, 176.30it/s]


In [185]:
# word mover distance residuals
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        df.loc[index, 'word_mover_distance_residuals_prediction'] = model.wmdistance(row['question_residuals'], row['prediction_sentence'])

calculating word mover distance for test: 100%|██████████| 74286/74286 [04:45<00:00, 260.07it/s]


In [ ]:
# for name, df in [('train', dftrain), ('test', dftest)]:
#     df['similarity_difference'] = df['similarity_tfid'] - df['similarity ']

In [ ]:
# # calculate similarity between prediction sentence and question 
# for name, df in [('train', dftrain), ('test', dftest)]:
#     for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
#         vectors = vectorizer.transform([row.question_residuals, row.paragraph])
#         df.loc[index, 'similarity_residuals_tfid'] = (vectors * vectors.T)[0, 1]

In [93]:
# calculate cummulative nlpS - split question to chunks
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
        question = row.question_lemmatized
        question_str = question
        question_len = len(question_str)
        similarities = []
        if int(question_len / 3) != 0:
            for i in range(0, question_len, int(question_len / 3)):
                question_chunk = question[i:i + int(question_len / 3)]
                vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
                similarities.append((vectors * vectors.T)[0, 1])
        elif int(question_len / 2) != 0:
            for i in range(0, question_len, int(question_len / 2)):
                question_chunk = question[i:i + int(question_len / 2)]
                vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
                similarities.append((vectors * vectors.T)[0, 1])
        else:
            question_chunk = question
            vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
            similarities.append((vectors * vectors.T)[0, 1])
        df.loc[index, 'similarity_tfid_split_max'] = np.max(similarities)
        df.loc[index, 'similarity_tfid_split_min'] = np.min(similarities)
        df.loc[index, 'similarity_tfid_split_mean'] = np.mean(similarities)
        df.loc[index, 'similarity_tfid_split_std'] = np.std(similarities)

calculating nlps for test: 100%|██████████| 74286/74286 [29:39<00:00, 41.74it/s]


**словосочетания**

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        question_patterns = list(get_pattern(row.question).keys())
        paragraph_patterns = list(get_pattern(row.paragraph).keys())
        vectorizer = TfidfVectorizer(ngram_range=(1,3))
        vectorizer.fit(np.append(question_patterns, paragraph_patterns))
        similarity_pattern = []
        for pattern in question_patterns:
            vectors = vectorizer.transform(np.append(pattern, paragraph_patterns))
            # eliminate diagonal 1'ses
            matrix = (vectors * vectors.T)
            n = matrix.shape[0]
            matrix[range(n), range(n)] = 0
            similarity_pattern.append(matrix[0].max())
        # get max similarity
        df.loc[index, 'similarity_pattern_mean'] = np.mean(similarity_pattern)
        df.loc[index, 'similarity_pattern_std'] = np.std(similarity_pattern)
        if len(similarity_pattern) == 0:
            df.loc[index, 'similarity_pattern_max'] = 0
            df.loc[index, 'similarity_pattern_min'] = 0
        else:
            df.loc[index, 'similarity_pattern_max'] = np.max(similarity_pattern)
            df.loc[index, 'similarity_pattern_min'] = np.min(similarity_pattern)
    df.fillna(0, inplace = True)

grammar for train:   9%|▉         | 14692/159391 [28:35<4:19:53,  9.28it/s] 

### Unique words

In [94]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        question_lemmatized = row.question_lemmatized
        
        unique_words_len = len(set(question_lemmatized))
        df.loc[index, 'unique_words_relative_to_question'] = unique_words_len / row.len_question
        
        word_occurences = []
        for word in question_lemmatized.split(' '):
            word_occurences.append(question_lemmatized.count(word))
            
        df.loc[index, 'word_ocurences_mean'] = np.mean(word_occurences)
        df.loc[index, 'word_ocurences_std'] = np.std(word_occurences)
        df.loc[index, 'word_ocurences_max'] = np.max(word_occurences)
        df.loc[index, 'word_ocurences_max_relative_to_question'] = np.max(word_occurences) / row.len_question
        

grammar for test: 100%|██████████| 74286/74286 [04:14<00:00, 292.26it/s]


## Ручные правила

question_types_keys = ['кто', 'кого', 'когда', 'почему', 'зачем', 'отчего', 'куда', 'кому', 'чему', 'как', 'о ком', 'чем',
                  'на чем', 'как', 'сколько', 'где', 'какого', 'откуда', 'чего', 'кем', 'чем', 'чей', 'который', 'какой',
                  'какая', 'какие', 'какое', 'каких', 'какие', 'чья', 'чьи', 'чье']

In [187]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        
        question_words = row.question.lower().strip('?').split(' ')
        paragraph_words = row.paragraph.lower().replace('.', '').split(' ')
        
        count = 0
        for word in question_words:
            if word in paragraph_words:
                count += 1
                
        df.loc[index, 'question_words_in_paragraph_relative'] = count / len(question_words)

grammar for test: 100%|██████████| 74286/74286 [01:03<00:00, 1163.37it/s]


In [197]:
# relative siimlarities
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        
        similarity_tfid = row.similarity_tfid
        similarity_tfid_split_mean = row.similarity_tfid_split_mean
        similarity_pattern_mean = row.similarity_pattern_mean
        
        word_mover_distance = row.word_mover_distance
        word_mover_sentence_distance_mean = row.word_mover_sentence_distance_mean
        word_mover_sentence_distance_split_mean = row.word_mover_sentence_distance_split_mean
        word_mover_sentence_distance_split_lemm_mean = row.word_mover_sentence_distance_split_lemm_mean
        word_mover_distance_residuals = row.word_mover_distance_residuals
        word_mover_distance_residuals_prediction = row.word_mover_distance_residuals_prediction
        
        df.loc[index, 'similarity_all_mean'] = np.mean([similarity_tfid, similarity_tfid_split_mean, similarity_pattern_mean])
        
        if similarity_tfid_split_mean != 0:
            df.loc[index, 'similarity_tfid_relative_split'] = similarity_tfid / similarity_tfid_split_mean
        else:
            df.loc[index, 'similarity_tfid_relative_split'] = 0
            
        if similarity_pattern_mean != 0:
            df.loc[index, 'similarity_tfid_relative_pattern'] = similarity_tfid / similarity_pattern_mean
        else:
            df.loc[index, 'similarity_tfid_relative_pattern'] = 0
            
        # w2v
        df.loc[index, 'word_mover_distance_all_mean'] = np.mean([word_mover_distance, 
                                                                word_mover_sentence_distance_mean, 
                                                               word_mover_sentence_distance_split_mean,
                                                               word_mover_sentence_distance_split_lemm_mean])
        df.loc[index, 'word_mover_distance_all_mean_all'] = np.mean([word_mover_distance, 
                                                                word_mover_sentence_distance_mean, 
                                                               word_mover_sentence_distance_split_mean,
                                                               word_mover_sentence_distance_split_lemm_mean,
                                                                   word_mover_distance_residuals,
                                                                   word_mover_distance_residuals_prediction])
        
        if word_mover_sentence_distance_mean != 0:
            df.loc[index, 'word_mover_distance_relative_sentence'] = word_mover_distance / word_mover_sentence_distance_mean
        else:
            df.loc[index, 'word_mover_distance_relative_sentence'] = 0
            
        if word_mover_sentence_distance_split_mean != 0:
            df.loc[index, 'word_mover_distance_relative_sentence_split'] = word_mover_distance / word_mover_sentence_distance_split_mean
        else:
            df.loc[index, 'word_mover_distance_relative_sentence_split'] = 0
            
        if word_mover_distance_residuals_prediction != 0:
            df.loc[index, 'word_mover_distance_residuals_relative_paragraph'] = word_mover_distance_residuals / word_mover_distance_residuals_prediction
        else:
            df.loc[index, 'word_mover_distance_residuals_relative_paragraph'] = 0
            
        
            

grammar for test: 100%|██████████| 74286/74286 [06:23<00:00, 193.93it/s]


In [203]:
dftrain.tail()

,generated_p_3,generated_p_6,paragraph,paragraph_id,question,question_id,target,len_paragraph,len_question,len_intersection,idf_question,idf_paragraph,idf_intersection,relative_question_len,relative_intersection_len,relative_intersection_question_len,relative_question_idf,relative_intersection_idf,relative_intersection_question_idf,word_idf_paragraph,word_idf_question,word_idf_intersection,paragraph_lemmatized,question_lemmatized,word2vec_similarity_max,word2vec_similarity_min,word2vec_similarity_mean,word2vec_similarity_std,len_intersection_w2v,idf_intersection_w2v,relative_intersection_len_w2v,relative_intersection_question_len_w2v,relative_intersection_idf_w2v,relative_intersection_question_idf_w2v,word_idf_intersection_w2v,word_mover_distance,word_mover_distance_lemm,word_mover_sentence_distance_mean,word_mover_sentence_distance_std,word_mover_sentence_distance_max,word_mover_sentence_distance_min,word_mover_sentence_distance_split_max,word_mover_sentence_distance_split_min,word_mover_sentence_distance_split_mean,word_mover_sentence_distance_split_std,word_mover_sentence_distance_split_lemm_max,word_mover_sentence_distance_split_lemm_min,word_mover_sentence_distance_split_lemm_mean,word_mover_sentence_distance_split_lemm_std,answer,...,question_residuals_len_relative_prediction_sentence,question_residuals_idf,question_residuals_idf_relative_question,question_residuals_idf_relative_prediction_sentence,overlap,len_overlap_relative_paragraph_max,len_overlap_relative_question_max,idf_overlap_max,len_overlap_relative_paragraph_mean,len_overlap_relative_question_mean,idf_overlap_mean,len_overlap_relative_paragraph_std,len_overlap_relative_question_std,idf_overlap_std,similarity_tfid,word_mover_distance_prediction,similarity_prediction_tfid,similarity_tfid_split_max,similarity_tfid_split_min,similarity_tfid_split_mean,similarity_tfid_split_std,unique_words_relative_to_question,word_ocurences_mean,word_ocurences_std,word_ocurences_max,word_ocurences_max_relative_to_question,misspelling,uncategorized,whitespace,typographical,duplication,target_1,similarity_pattern_mean,similarity_pattern_std,similarity_pattern_max,similarity_pattern_min,word_mover_distance_residuals,similarity_residuals_tfid,word_mover_distance_residuals_prediction,question_words_in_paragraph_relative,similarity_all_mean,similarity_tfid_relative_split,similarity_tfid_relative_pattern,word_mover_distance_all_mean,word_mover_distance_all_mean_all,word_mover_distance_relative_sentence,word_mover_distance_relative_sentence_split,word_mover_distance_residuals_relative_paragraph,question_words_in_prediction_sentence_words_relative,question_words_count_relative
1480469,0.0,1.0,"Точное место погребения Моцарта доподлинно неизвестно: в его времена могилы оставались необозначенными, надгробные камни разрешалось ставить не на месте самого захоронения, а у стены кладбища. Могилу Моцарта много лет подряд навещала супруга его друга Иоганна Георга Альбрехтсбергера, которая брала с собой сына. Он точно помнил место захоронения композитора и, когда, по случаю пятидесятилетия со дня смерти Моцарта, стали разыскивать его захоронение, смог его показать. В 1859 году там соорудили памятник по проекту фон Гассера — знаменитого Плачущего Ангела. В связи со столетием со дня смерти композитора памятник перенесли в музыкальный уголок Центрального кладбища Вены, из-за чего снова возникла опасность потерять настоящую могилу. Тогда надзиратель кладбища Святого Марка Александр Кругер из различных остатков прежних надгробий соорудил маленький памятник. В настоящее время, Плачущий Ангел возвращён на своё прежнее место.",1520,Кого брала с Точное не навещала супруга ?,140847.0,0.0,104.0,7.0,6.0,50.650145,761.173083,44.633552,0.067308,0.057692,0.857143,0.066542,0.058638,0.881213,7.318972,7.235735,7.438925,точный место погребение моцарт доподлинно неизвестно время могила оставаться необозначенный надгробный камень разрешаться ставить место самый захоронение стена кладбище могила моцарт год подряд наве

In [204]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        
        question_words = row.question.lower().strip('?').split(' ')
        prediction_sentence_words = row.prediction_sentence.lower().replace('.', '').split(' ')
        
        count = 0
        for word in question_words:
            if word in prediction_sentence_words:
                count += 1
        
        df.loc[index, 'question_words_in_prediction_sentence_words_relative'] = count / len(question_words)
        
    df.loc[:, 'question_words_count_relative'] = df['question_words_in_prediction_sentence_words_relative'] / df['question_words_in_paragraph_relative']


grammar for test: 100%|██████████| 74286/74286 [00:57<00:00, 1283.33it/s]


## Сохраним данные

In [205]:
dftrain.to_csv('../../data/train_task1_latest_pred.csv', sep=';', index= False)
dftest.to_csv('../../data/test_task1_latest_pred.csv', sep=';', index= False)

## Загрузим даные

In [11]:
dftrain = pd.read_csv('../../data/train_task1_latest_pred.csv', sep=';')
dftest = pd.read_csv('../../data/test_task1_latest_pred.csv', sep=';')

### Let us see

In [ ]:
pd.options.display.max_colwidth = 10000

### Patterns